In [1]:
from getsparkcontext import sc
print(sc)

<SparkContext master=local appName=Application>


In [2]:
marvel_graph_rdd = sc.textFile('file:///Users/hdagar3/Documents/Spark_Things/Spark_Course_Files/Marvel-Graph.txt')

In [3]:
print(marvel_graph_rdd)

file:///Users/hdagar3/Documents/Spark_Things/Spark_Course_Files/Marvel-Graph.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0


In [4]:
print(marvel_graph_rdd.count())

6589


In [6]:
# print(marvel_graph_rdd.collect())

In [7]:
'''
In Spark you must know about the elements of RDD, i.e whenever you apply map operation on RDD then one by one, all
elements get processed in it.
Generally we prefer element of RDD to be a string or tuple or list --> so whenever we are applying map operation then
you would get either string(very initial) or tuple(after processing) or list(rare)


Whenever we use flatMap() on rdd then try to collect items in list then after processing, each item in list would 
become the element of RDD i.e after returning the list, all elements of list would become elements of RDD . 
--> V.V. IM Concept

'''

'\nIn Spark you must know about the elements of RDD, i.e whenever you apply map operation on RDD then one by one, all\nelements get processed in it.\nGenerally we prefer element of RDD to be a string or tuple or list --> so whenever we are applying map operation then\nyou would get either string(very initial) or tuple(after processing) or list(rare)\n\n\nWhenever we use flatMap() on rdd then try to collect items in list then after processing, each item in list would \nbecome the element of RDD i.e after returning the list, all elements of list would become elements of RDD . \n--> V.V. IM Concept\n\n'

In [102]:
import sys
start_superhero_id = 5
end_superhero_id = 14

print(sys.maxsize)

9223372036854775807


In [103]:
def makeAdjacencyList(line):
    info = line.split()
    super_hero_id = int(info[0])
    
    links = list()
    
    for item in info[1:]:
        links.append(int(item))
    
    color = 'WHITE'
    distance = sys.maxsize
    
    if super_hero_id == start_superhero_id:
        color = 'GRAY'
        distance = 0
    
    return (super_hero_id,(links,distance,color))

In [104]:
adjacency_list_rdd = marvel_graph_rdd.map(makeAdjacencyList)
# adjacency_list_rdd.count()

In [105]:
# whenever this count becomes 1 then stop searching further, we have found end_superhero_id
# it is accumulator i.e it will be shared among all the executors i.e all the worker nodes.
count = sc.accumulator(0)

In [106]:
def process_each_element(line):
    superhero_id = line[0]
    links_list = line[1][0]
    distance = line[1][1]
    color = line[1][2]
    
    list_of_elements = list()
    if color == 'GRAY':
        if(superhero_id == end_superhero_id):
                count.add(1)
                
        for item in links_list:
            list_of_elements.append((item,([],distance+1,'GRAY')))
        
        color = 'BLACK'
    list_of_elements.append(line)
    
    return list_of_elements            

In [107]:
def reducer_of_superheros(tuple1,tuple2):
    
    links1 = tuple1[0]
    distance1 = tuple1[1]
    color1 = tuple1[2]
    
    links2 = tuple2[0]
    distance2 = tuple2[1]
    color2 = tuple2[2]
    
    
    # always try to contruct list separately instead of mutating existing.
    final_links = list()
        
    if len(links1) > 0:
        final_links.extend(links1)
    if len(links2) > 0:
        final_links.extend(links2)
        
    # links1.extend(links2)
    # final_links = links1

    
    final_distance = min(distance1,distance2)
    
    final_color = None
    
    if color1 == 'WHITE':
        final_color = color2
    
    elif color1 == 'GRAY':
        if color2 == 'BLACK':
            final_color = color2
        else:
            final_color = color1
    else:
        final_color = color1
        
    return (final_links,final_distance,final_color)

In [108]:
# no of iterations for mapper and reducer

degree_of_separation = -1

for i in range(10):
    
    print(f"processing iteration #{i}")
    
    degree_of_separation += 1

    
    mapped_reducable_rdd = adjacency_list_rdd.flatMap(process_each_element)

    # In this for loop, you just applied transformation, no action is there so above transformations won't 
    # get executed. In order to execute that, I am applying this action
    print(mapped_reducable_rdd.count())
    
    if(count.value > 0):
        break
        
    # this adjacency_list_rdd name must be same as the name of above adjacency_list_rdd, so that we can continue the
    # process on modified rdd --> V. IM
    adjacency_list_rdd = mapped_reducable_rdd.reduceByKey(reducer_of_superheros)
    
    
print(degree_of_separation)

processing iteration #0
6597
processing iteration #1
10940
processing iteration #2
221938
processing iteration #3
342607
3


In [87]:
a = [1,2,3,4]
a.extend([7,8,9])

A = []
B = [1,2,3,4]

A.extend(B)
print(A)

[1, 2, 3, 4]


In [ ]:
# END